# Optimisation Case Study - Staff Planning

An insurance company InsurePlus wants us to help them with finding the optimal number of staff 
for their insurance application approval process for the calendar year 2021.

Steps:     
1 Reading data from Excel workbook   
2 Data Preprocessing    
3 Model bulding    
4 Task related to Questions  

Submitted by
Aravind Girish, Thera Chandra & Divya Saurabh


In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook

Inputdata = "Staffing+Data.xlsx"
StaffAv = pd.read_excel(Inputdata, sheet_name ='StaffAvailability') # Create a Dataframe for the Staff Availability
Cost = pd.read_excel(Inputdata, sheet_name ='Cost') # Create a Dataframe for the Staffing Cost 
transcost = pd.read_excel(Inputdata, sheet_name ='ServiceRate')# Create a Dataframe for the Service Rate 
MgDemandDist = pd.read_excel(Inputdata, sheet_name ='DemandData')# Create a Dataframe for the Demand 


## Data pre-processing 

In [4]:
# Create the required Python data structures for indexes and parameters

# Demand Data
Demand = MgDemandDist.set_index(['State','Month'])['Demand'].to_dict()

# FTE Salray from data 
FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Unit Outsourcing Cost
UnitOutsourcingCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of applcations when Employee working Full Time 
FTE_AppServeRate = transcost.iloc[0,0]

# Staff Availablility for serving Insurance Application
StaffAvPer = StaffAv.set_index(['State','Month'])['StaffAvPer'].to_dict()
StaffAv_LB = StaffAv.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = StaffAv.set_index(['State','Month'])['UB'].to_dict()

# Index
BankLoc = Cost['State'].unique()
Month = Cost['Month'].unique()


In [5]:
StaffAv.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [6]:
Month

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

In [7]:
BankLoc

array(['A', 'B', 'C'], dtype=object)

In [8]:
BankLoc.tolist()

['A', 'B', 'C']

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



This part is coverd in report Document

**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [9]:
# Creating a model instance
model = ConcreteModel()

In [10]:
model.i = Set(initialize=BankLoc.tolist(), doc = 'States') 
model.j = Set(initialize=Month.tolist(), doc = 'Months') 

In [11]:
# Define Pyomo sets and Parameters
model.d = Param(model.i,model.j,initialize = Demand, doc='Demand')
model.s = Param(model.i,model.j,initialize = FTE_Salary, doc='FTE Salary')
model.oc = Param(model.i,model.j,initialize = UnitOutsourcingCost, doc='UnitOutSourceCost')
model.sa = Param(model.i,model.j,initialize = StaffAvPer, doc = 'StaffAvPercent')

model.r = Param(initialize = FTE_AppServeRate, doc = 'FTE App Serv Rate')

In [12]:
# No of Appl that can be processed for the given staff av
def c_FTEAPP(model,i,j):
    return model.r*model.sa[i,j]

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc = 'No of FTE Application' )

In [13]:
# Reg_A - Restriction for A city
def Reg_A(model,i,j):
    return 0.30*model.d[i,j]

model.Reg_A= Param(model.i, model.j, initialize=Reg_A, doc = 'RegRest_A')

In [14]:
# Reg_B- - Restriction for A city
def Reg_B(model,i,j):
    return 0.40*model.d[i,j]

model.Reg_B= Param(model.i, model.j, initialize=Reg_B, doc = 'RegRest_B')

In [15]:
# Decision variables
model.x = Var(model.i, model.j, domain = NonNegativeReals, doc='No of FTE',)
model.y = Var(model.i, model.j, domain = NonNegativeIntegers, doc='No of outsource App')

In [16]:
# Constraints
# Demand Constraints
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr=model.x[i,j] * model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [17]:
# Regulalatory Constraints of City 'A'-30% and City 'B'-40%
model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i=='A':
            model.regulatoryCons.add(expr =model.y['A',j]<=model.Reg_A['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr =model.y['B',j]<=model.Reg_B['B',j])
            

In [18]:
# Objective function

def objective_rule(model):
    return (sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) + 
               sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j))

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')


In [19]:
# Invoking the solver

result_cost = SolverFactory('glpk').solve(model)
result_cost.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.056127071380615234
# ----------------------------------------------------------
#   Solution Information
# -----------

In [20]:
#Display the val of var A for Dec
print(model.x['A','Dec'].value) #139
print(model.y['A','Dec'].value) #1554

139.576923076923
1554.0


In [21]:
# Print the value of the objective function

Total_Cost = model.objective.expr()
print(Total_Cost)


17962336.448769882


**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [22]:
# Creating dataframe for the results

FTE_staff = []
for i in BankLoc:
    for j in Month:
        v = model.x[i,j].value # FTE
        w = model.y[i,j].value # App_outsourced
        FTE_staff.append([i,j,v,w])

FTE_staff_AV = pd.DataFrame(FTE_staff, columns=['State', 'Month', 'FTE', 'Outsourced'])

In [23]:
FTE_staff_AV

,State,Month,FTE,Outsourced
0,A,Jan,161.728395,0.0
1,A,Feb,160.460526,0.0
2,A,Mar,198.066667,0.0
3,A,Apr,71.781250,0.0
4,A,May,63.846154,0.0
5,A,Jun,77.910959,0.0
6,A,Jul,137.279412,1600.0
7,A,Aug,110.888158,0.0
8,A,Sep,116.018519,0.0
9,A,Oct,120.856164,0.0


In [24]:
# Creating column ID for unique identification
FTE_staff_AV['ID'] = FTE_staff_AV.State + '-' + FTE_staff_AV.Month
FTE_staff_AV['Demand'] = MgDemandDist['Demand']
FTE_staff_AV['StaffAvPer'] = StaffAv['StaffAvPer']
FTE_staff_AV['FTE Applications'] = FTE_staff_AV.StaffAvPer * FTE_staff_AV.FTE * 40
FTE_staff_AV

,State,Month,FTE,Outsourced,ID,Demand,StaffAvPer,FTE Applications
0,A,Jan,161.728395,0.0,A-Jan,5240,0.81,5240.0
1,A,Feb,160.460526,0.0,A-Feb,4878,0.76,4878.0
2,A,Mar,198.066667,0.0,A-Mar,5942,0.75,5942.0
3,A,Apr,71.781250,0.0,A-Apr,2297,0.80,2297.0
4,A,May,63.846154,0.0,A-May,1992,0.78,1992.0
5,A,Jun,77.910959,0.0,A-Jun,2275,0.73,2275.0
6,A,Jul,137.279412,1600.0,A-Jul,5334,0.68,3734.0
7,A,Aug,110.888158,0.0,A-Aug,3371,0.76,3371.0
8,A,Sep,116.018519,0.0,A-Sep,3759,0.81,3759.0
9,A,Oct,120.856164,0.0,A-Oct,3529,0.73,3529.0


In [25]:
# Cost per application
AvgCost = Total_Cost / MgDemandDist['Demand'].sum()
AvgCost

158.5462288273861

In [26]:
# Writing the results in to an Excel sheet
FTE_staff_AV.to_excel('FTEstaffAv.xlsx', sheet_name='FTEstaffAv')

# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [27]:
# Creating a model instance
model = ConcreteModel()

In [28]:
model.i = Set(initialize=BankLoc.tolist(), doc = 'States') 
model.j = Set(initialize=Month.tolist(), doc = 'Months') 

In [29]:
# Define Pyomo sets and Parameters
model.d = Param(model.i,model.j,initialize = Demand, doc='Demand')
model.s = Param(model.i,model.j,initialize = FTE_Salary, doc='FTE Salary')
model.oc = Param(model.i,model.j,initialize = UnitOutsourcingCost, doc='UnitOutSourceCost')
model.salb = Param(model.i,model.j,initialize = StaffAv_LB, doc = 'StaffAvPercent')

model.r = Param(initialize = FTE_AppServeRate, doc = 'FTE App Serv Rate')

In [30]:
# No of Appl that can be processed for the given staff av
def c_FTEAPP(model,i,j):
    return model.r*model.salb[i,j]

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc = 'No of FTE Application' )

In [31]:
# Reg_A - Restriction for A city
def Reg_A(model,i,j):
    return 0.30*model.d[i,j]

model.Reg_A= Param(model.i, model.j, initialize=Reg_A, doc = 'RegRest_A')

In [32]:
# Reg_B- - Restriction for A city
def Reg_B(model,i,j):
    return 0.40*model.d[i,j]

model.Reg_B= Param(model.i, model.j, initialize=Reg_B, doc = 'RegRest_B')

In [33]:
# Decision variables
model.x = Var(model.i, model.j, domain = NonNegativeReals, doc='No of FTE',)
model.y = Var(model.i, model.j, domain = NonNegativeIntegers, doc='No of outsource App')

In [34]:
# Constraints
# Demand Constraints
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr=model.x[i,j] * model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [35]:
# Regulalatory Constraints of City 'A'-30% and City 'B'-40%
model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i=='A':
            model.regulatoryCons.add(expr =model.y['A',j]<=model.Reg_A['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr =model.y['B',j]<=model.Reg_B['B',j])
            

In [36]:
# Objective function

def objective_rule(model):
    return (sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) + 
               sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j))

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')


In [37]:
# Invoking the solver

result_cost = SolverFactory('glpk').solve(model)
result_cost.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05768847465515137
# ----------------------------------------------------------
#   Solution Information
# ------------

In [38]:
# Print the value of the objective function

Total_Cost = model.objective.expr()
print(Total_Cost)


19599482.516788766


**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [39]:
# Creating dataframe for the results

FTE_staff = []
for i in BankLoc:
    for j in Month:
        v = model.x[i,j].value # FTE
        w = model.y[i,j].value # App_outsourced
        FTE_staff.append([i,j,v,w])

FTE_staff_LB = pd.DataFrame(FTE_staff, columns=['State', 'Month', 'FTE', 'Outsourced'])

In [40]:
FTE_staff_LB

,State,Month,FTE,Outsourced
0,A,Jan,187.142857,0.0
1,A,Feb,131.346154,1463.0
2,A,Mar,212.214286,0.0
3,A,Apr,76.566667,0.0
4,A,May,71.142857,0.0
5,A,Jun,61.269231,682.0
6,A,Jul,155.583333,1600.0
7,A,Aug,90.769231,1011.0
8,A,Sep,134.250000,0.0
9,A,Oct,95.038462,1058.0


In [41]:
# Optimal number of staff members Avg

optimal_staff_members_Avg = round(FTE_staff_LB['FTE'].mean(),1)
optimal_staff_members_Avg

76.7

In [42]:
# Optimal number of staff members
FTE_staff_LB['FTE'].sum()

2761.2709706959704

#### 3.1.2 Percentage of outsourced applications 

In [43]:
# write your code here
TotalDemand = MgDemandDist['Demand'].sum()
PerOutsource = (FTE_staff_LB['Outsourced'].sum() / TotalDemand) * 100
PerOutsource

35.13690045368687

#### 3.1.3 Average cost per application

In [44]:
# write your code here
AvgCost = Total_Cost / TotalDemand
AvgCost

172.9966504562357

In [45]:
# Creating column ID for unique identification
FTE_staff_LB['ID'] = FTE_staff_LB.State + '-' + FTE_staff_LB.Month
FTE_staff_LB['Demand'] = MgDemandDist['Demand']
FTE_staff_LB['LB'] = StaffAv['LB']
FTE_staff_LB['FTE Applications'] = FTE_staff_LB.LB * FTE_staff_LB.FTE * 40

# Writing the results in to an Excel sheet
FTE_staff_LB.to_excel('FTEstaffLB.xlsx', sheet_name='FTEstafLB')

### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [46]:
# Creating a model instance


In [47]:
# Creating a model instance
model = ConcreteModel()

In [48]:
model.i = Set(initialize=BankLoc.tolist(), doc = 'States') 
model.j = Set(initialize=Month.tolist(), doc = 'Months') 

In [49]:
# Define Pyomo sets and Parameters
model.d = Param(model.i,model.j,initialize = Demand, doc='Demand')
model.s = Param(model.i,model.j,initialize = FTE_Salary, doc='FTE Salary')
model.oc = Param(model.i,model.j,initialize = UnitOutsourcingCost, doc='UnitOutSourceCost')
model.saub = Param(model.i,model.j,initialize = StaffAv_UB, doc = 'StaffAvPercent')

model.r = Param(initialize = FTE_AppServeRate, doc = 'FTE App Serv Rate')

In [50]:
# No of Appl that can be processed for the given staff av
def c_FTEAPP(model,i,j):
    return model.r*model.saub[i,j]

model.FTEAPPNO = Param(model.i, model.j, initialize=c_FTEAPP, doc = 'No of FTE Application' )

In [51]:
# Reg_A - Restriction for A city
def Reg_A(model,i,j):
    return 0.30*model.d[i,j]

model.Reg_A= Param(model.i, model.j, initialize=Reg_A, doc = 'RegRest_A')

In [52]:
# Reg_B- - Restriction for A city
def Reg_B(model,i,j):
    return 0.40*model.d[i,j]

model.Reg_B= Param(model.i, model.j, initialize=Reg_B, doc = 'RegRest_B')

In [53]:
# Decision variables
model.x = Var(model.i, model.j, domain = NonNegativeReals, doc='No of FTE',)
model.y = Var(model.i, model.j, domain = NonNegativeIntegers, doc='No of outsource App')

In [54]:
# Constraints
# Demand Constraints
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr=model.x[i,j] * model.FTEAPPNO[i,j] + model.y[i,j] == model.d[i,j])

In [55]:
# Regulalatory Constraints of City 'A'-30% and City 'B'-40%
model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i=='A':
            model.regulatoryCons.add(expr =model.y['A',j]<=model.Reg_A['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr =model.y['B',j]<=model.Reg_B['B',j])
            

In [56]:
# Objective function

def objective_rule(model):
    return (sum(model.s[i,j] * model.x[i,j] for i in model.i for j in model.j) + 
               sum(model.oc[i,j] * model.y[i,j] for i in model.i for j in model.j))

model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')


In [57]:
# Invoking the solver

result_cost = SolverFactory('glpk').solve(model)
result_cost.write()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.061115264892578125
# ----------------------------------------------------------
#   Solution Information
# -----------

In [58]:
# Print the value of the objective function

Total_Cost = model.objective.expr()
print(Total_Cost)


16527535.63793573


**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [59]:
# Creating dataframe for the results

FTE_staff = []
for i in BankLoc:
    for j in Month:
        v = model.x[i,j].value # FTE
        w = model.y[i,j].value # App_outsourced
        FTE_staff.append([i,j,v,w])

FTE_staff_UB = pd.DataFrame(FTE_staff, columns=['State', 'Month', 'FTE', 'Outsourced'])

In [60]:
FTE_staff_UB

,State,Month,FTE,Outsourced
0,A,Jan,145.555556,0.0
1,A,Feb,143.470588,0.0
2,A,Mar,185.687500,0.0
3,A,Apr,67.558824,0.0
4,A,May,58.588235,0.0
5,A,Jun,71.093750,0.0
6,A,Jul,177.800000,0.0
7,A,Aug,99.147059,0.0
8,A,Sep,104.416667,0.0
9,A,Oct,110.281250,0.0


In [61]:
# Optimal number of staff members Avg

optimal_staff_members_Avg = round(FTE_staff_UB['FTE'].mean(),1)
optimal_staff_members_Avg

92.9

In [62]:
# Optimal number of staff members
FTE_staff_UB['FTE'].sum()

3343.189524976658

#### 3.2.2 Percentage of outsourced applications

In [63]:
# write your code here
TotalDemand = MgDemandDist['Demand'].sum()
PerOutsource = (FTE_staff_UB['Outsourced'].sum() / TotalDemand) * 100
PerOutsource

4.106130951330168

#### 3.2.3 Average cost per application

In [64]:
# write your code here
AvgCost = Total_Cost / TotalDemand
AvgCost

145.88182638035315

In [65]:
# Creating column ID for unique identification
FTE_staff_UB['ID'] = FTE_staff_UB.State + '-' + FTE_staff_UB.Month
FTE_staff_UB['Demand'] = MgDemandDist['Demand']
FTE_staff_UB['UB'] = StaffAv['UB']
FTE_staff_UB['FTE Applications'] = FTE_staff_UB.UB * FTE_staff_UB.FTE * 40

# Writing the results in to an Excel sheet
FTE_staff_UB.to_excel('FTEstaffUB.xlsx', sheet_name='FTEstafUB')

# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

This part is coverd in report Document